In [19]:
!pip install datasets
!pip install accelerate

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import torch
import random
import tensorflow as tf
from datasets import Dataset
import tqdm as notebook_tqdm
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, f1_score

df_test = pd.read_csv("politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_test['tweet'] = df_test.apply(lambda row: row['ideology_multiclass'] + " " + row['profession']+ " " + row['ideology_binary']+ " " + row['tweet'], axis=1)
df_test.drop(['ideology_binary','ideology_multiclass','profession','label'], axis=1, inplace=True)


In [21]:
df_test

,gender,tweet
0,male,"moderate_right journalist right Ayer, en Guada..."
1,male,"moderate_right journalist right “Chile Vamos”,..."
2,male,moderate_right journalist right Se está tratan...
3,male,"moderate_right journalist right @user Lucía, l..."
4,male,moderate_right journalist right JxC no dio quó...
...,...,...
43755,male,"right politician right No, si ahora resulta qu..."
43756,male,right politician right Tenemos un buen futuro ...
43757,male,right politician right Cuando el [POLITICAL_PA...
43758,male,right politician right La entrevista de @user ...


In [22]:
# Initialize the tokenizer
checkpoint_name= 'juan-glez29/BERTuit-gender-all'
tokenizer = AutoTokenizer.from_pretrained(checkpoint_name)
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_name)

In [23]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df_test['gender'] = label_encoder.fit_transform(df_test['gender'])

In [24]:
df_test['tweet'] = df_test['tweet'].apply(lambda x: x.replace("@user", "<usr>"))

In [25]:
def tokenize_function(example):
    return tokenizer(example['tweet'])

# Conversión del DataFrame de pandas a un Dataset de Hugging Face para pasarlo al modelo
huggingface_dataset = Dataset.from_pandas(df_test)
huggingface_dataset

Dataset({
    features: ['gender', 'tweet'],
    num_rows: 43760
})

In [26]:
# Aplicamos la tokenización a todo el dataset
tokenized_dataset = huggingface_dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns("tweet")
test_dataset = tokenized_dataset.rename_column("gender", "labels")
test_dataset

Map:   0%|          | 0/43760 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 43760
})

In [27]:
# Define test trainer
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
test_trainer = Trainer(model, data_collator=data_collator, tokenizer=tokenizer)

predicted_results = test_trainer.predict(test_dataset)

# Preprocess raw predictions
y_pred = predicted_results.predictions.argmax(-1) # Get the highest probability prediction

labels = test_dataset["labels"]

macro_f1 = f1_score(labels, y_pred, average='macro')# o weighted si estan desbalceadas las clases

print(f"F1-Score:\t{macro_f1}")

F1-Score:	0.5914822848348402
